# Database
The following notebook build a sqlite database of approximately 9,000 earthquakes and ~200 countries. This db is later used to create 5 visualisations.

The table **countries** has the following columns:

1. Country Code
2. Country Name
3. Total Population (millions) ^
4. Human Developmnet Index ^

The table **earthquakes** was built using a GeoJSON file with information of every earthquake in the past 30 days. This dataset belongs to the United States Geological Survey (USGS)^^ and includes the following columns:
1. USGS Earthquake ID
2. Longitude
3. Latitude
4. Depth
5. Timestamp
6. Magnitude
7. Country Code

^ *Total Population* and *Human Development Index (HDI)* are values obtained from the *Human Development Report Office (HDRO)* (http://ec2-54-174-131-205.compute-1.amazonaws.com/API/Login.php). The HDRO offers an API for developers to query human development related data in JSON format. This dta is updated annually, upong the publication of the *global Human Development Report*

^^ USGS - GeoJSON Summary Format: https://earthquake.usgs.gov/earthquakes/feed/v1.0/geojson.php

Human development classification
HDI classifications are based on HDI fixed cutoff points, which are derived from the quartiles of dis
tributions of the component indicators. The cutoffpoints are HDI of less than 0.550 for low human development,
0.550–0.699 for medium human development, 0.700–0.799 for high human development and 0.800 or greater 
for very high human development.
Codes:
44206 - Total Population (millions)
137506 Human Development Index (HDI)

## Part 1: Importing/requesting data
Requesting json/geojson files from the HDRO and USGS.

In [1]:
import requests

In [2]:
url_HDRO = "http://ec2-54-174-131-205.compute-1.amazonaws.com/API/HDRO_API.php/indicator_id=137506,44206/year=2019"
url_USGS = "https://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/all_month.geojson"

response_HDRO = requests.get(url_HDRO)
print(f"HDRO: {response_HDRO}")
response_USGS = requests.get(url_USGS)
print(f"USGS: {response_USGS}")

HDRO: <Response [200]>
USGS: <Response [200]>


Saving the database into variables

In [3]:
data_HDRO = response_HDRO.json()
# print(json.dumps(data_HDRO, indent=4, sort_keys=True))
data_USGS = response_USGS.json()

The table *countries* required the country code in 2-digits-alpha format, but raw data was provided in 3-digit-alpha format. The repository https://github.com/lukes/ISO-3166-Countries-with-Regional-Codes includes a ready-to-use csv file with both, 2- and 3-digit country codes. This table was used to convert between the 2 formats.

Reading the csv file with country code information. 

In [4]:
import pandas as pd

countryCodes_df = pd.read_csv("../../data/all.csv")

## Part 2: Cleaning the data
**Actions taken to clean the csv file:**
1. The only columns needed are *Country name*, *country code in 2-digit format*, and *country code in 3-digit format*.
2. The 2-digit alpha code for country Namibia was missing, so the value was written manually.

In [5]:
# countryCodes_df
clean_cc_df = countryCodes_df.copy()[["name", "alpha-2", "alpha-3"]]
clean_cc_df['alpha-2'][153] = 'NA'
clean_cc_df['alpha-2'][153]

'NA'

In [6]:
clean_cc_df.to_csv("./output_data/clean_cc_df.csv")

Understanding the structure of the HDRO json file:

In [7]:
df_HDRO = pd.read_json(url_HDRO)
df_HDRO = df_HDRO.reset_index()
df_HDRO

,index,indicator_value,country_name,indicator_name
0,AFG,"{'44206': {'2019': 38.042}, '137506': {'2019':...",Afghanistan,NaN
1,AGO,"{'44206': {'2019': 31.825}, '137506': {'2019':...",Angola,NaN
2,ALB,"{'44206': {'2019': 2.8810000000000002}, '13750...",Albania,NaN
3,AND,"{'44206': {'2019': 0.077}, '137506': {'2019': ...",Andorra,NaN
4,ARE,"{'44206': {'2019': 9.771}, '137506': {'2019': ...",United Arab Emirates,NaN
...,...,...,...,...
192,ZAF,"{'44206': {'2019': 58.558}, '137506': {'2019':...",South Africa,NaN
193,ZMB,"{'44206': {'2019': 17.861}, '137506': {'2019':...",Zambia,NaN
194,ZWE,"{'44206': {'2019': 14.645}, '137506': {'2019':...",Zimbabwe,NaN
195,44206,NaN,NaN,Total population (millions)


Extracting the required values from the HDRO json file and saving them in a pandas dataframe using the try/except methods.

A few number of countries couldn't be saved into the dataframe as some of the required values were missing (see print outputs).

In [8]:
clean_HDRO_df = pd.DataFrame(columns=['CountryName', 'CountryCode', 'HDI', "TotalPopulation"]) 
# clean_HDRO_df
for index, country in df_HDRO.iterrows():
    try:
        value_totalPopulation = float(country["indicator_value"]['44206']['2019'])
        value_HDI = float(country["indicator_value"]['137506']['2019'])
        clean_HDRO_df = clean_HDRO_df.append([{"CountryName": country["country_name"], "CountryCode": country["index"], "HDI": value_HDI, "TotalPopulation": value_totalPopulation}], ignore_index=True)
    except:
        print(country["index"], country["indicator_value"])

clean_HDRO_df


MCO {'44206': {'2019': 0.039}}
NRU {'44206': {'2019': 0.011}}
PRK {'44206': {'2019': 25.666}}
SMR {'44206': {'2019': 0.034}}
SOM {'44206': {'2019': 15.443}}
TUV {'44206': {'2019': 0.012}}
44206 nan
137506 nan


,CountryName,CountryCode,HDI,TotalPopulation
0,Afghanistan,AFG,0.511,38.042
1,Angola,AGO,0.581,31.825
2,Albania,ALB,0.795,2.881
3,Andorra,AND,0.868,0.077
4,United Arab Emirates,ARE,0.890,9.771
...,...,...,...,...
184,Samoa,WSM,0.715,0.197
185,Yemen,YEM,0.470,29.162
186,South Africa,ZAF,0.709,58.558
187,Zambia,ZMB,0.584,17.861


In [9]:
clean_HDRO_df.to_csv("./output_data/clean_HDRO_df.csv")

The df *countries* and the df *country codes* were merged through the 3-digit country code columns:

In [10]:
clean_HDRO_df = clean_HDRO_df.merge(clean_cc_df, left_on="CountryCode", right_on="alpha-3", how="inner")
clean_HDRO_df =clean_HDRO_df.drop(["CountryCode"], axis=1)


**Final HDRO table**

In [11]:
clean_HDRO_df

,CountryName,HDI,TotalPopulation,name,alpha-2,alpha-3
0,Afghanistan,0.511,38.042,Afghanistan,AF,AFG
1,Angola,0.581,31.825,Angola,AO,AGO
2,Albania,0.795,2.881,Albania,AL,ALB
3,Andorra,0.868,0.077,Andorra,AD,AND
4,United Arab Emirates,0.890,9.771,United Arab Emirates,AE,ARE
...,...,...,...,...,...,...
184,Samoa,0.715,0.197,Samoa,WS,WSM
185,Yemen,0.470,29.162,Yemen,YE,YEM
186,South Africa,0.709,58.558,South Africa,ZA,ZAF
187,Zambia,0.584,17.861,Zambia,ZM,ZMB


In [12]:
clean_HDRO_df.to_csv("./output_data/clean_HDRO_df_MERGED.csv")

## Part 3: SQL database

Importing required SQL Alchemy modules:

In [13]:
# Import SQL Alchemy
from sqlalchemy import create_engine

# Import module to create relationship patterns
from sqlalchemy.orm import relationship

# Import and establish Base for which classes will be constructed
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()

# Import modules to declare columns and column data types
from sqlalchemy import Table, Column, Integer, String, Float, BigInteger, ForeignKey

Creating *Countries* and *Earthquakes* classes:

In [14]:
# Create classes:
# Countries & Earthquakes

class Countries(Base):  #Parent
    __tablename__ = 'countries'
    # id = Column(Integer, primary_key=True)
    CountryCode = Column(String(255), primary_key=True)
    CountryName = Column(String(255))
    Population = Column(Float)  # Millions
    HDIndex = Column(Float) # Index
    earthquake = relationship("Earthquakes")

class Earthquakes(Base):    #Child
    __tablename__ = 'earthquakes'
    # id = Column(Integer, primary_key=True)
    EarthquakeID = Column(String(255), primary_key=True)
    Latitude = Column(Float)
    Longitude = Column(Float)
    Depth = Column(Float)
    DateTime = Column(BigInteger)
    Magnitude = Column(Float)
    # CountryCode = Column(String(255))   # Foreign Key?
    country_id = Column(String(255), ForeignKey('countries.CountryCode'))
    # parent = relationship("Countries", back_populates="children")


# print(data_USGS["features"])

Since the GeoJson file doesn't provide the country name where each earthquake hit, reverse geocoding was implemented using the python library **reverse_geocoder** (https://github.com/thampiman/reverse-geocoder).

The library **tqdm** provides a nice and ready-to-use progress bar.

In [15]:
import reverse_geocoder as rg
from time import sleep
from tqdm.auto import tqdm

The next block of code execute the following actions:
1. Creating Database Connection
2. Establishing connection to a sqlite database
3. Creating countries and Earthquakes tables

In [16]:
# Create Database Connection
# ---------------------------
# Establish Connection
engine = create_engine("sqlite:///project2_test.sqlite")
conn = engine.connect()

# Create Countries and Earthquakes tables within the database
Base.metadata.create_all(conn)

# To push the objects made and query the server we use a Session object
from sqlalchemy.orm import Session
session = Session(bind=engine)

Inserting entries to the Country table.

In [17]:
for index, country in tqdm(clean_HDRO_df.iterrows(), total=len(clean_HDRO_df)):
    # sleep(0.1)
    try:
        # Add new_country to the current session
        new_country = Countries(CountryCode=country["alpha-2"],CountryName=country["CountryName"],Population=country["TotalPopulation"],HDIndex=country["HDI"])
        session.add(new_country)

        # Commit objects to the database
        session.commit()
    except:
        print(country["alpha-2"], country["CountryName"], country["TotalPopulation"], country["HDI"])


100%|██████████| 189/189 [00:22<00:00,  8.47it/s]


Inserting entries to the Earthquake table (Reverse geocoding is used here) and after completion closing the connection to the sqlite database.

In [18]:
cont=1
# Create specific instances of the Countries and Earthquakes classes
for earthquake in tqdm(data_USGS["features"], total=len(data_USGS["features"])):
    # sleep(0.1)

    if cont<=10:
        try:
            lon = earthquake["geometry"]["coordinates"][0]
            lat = earthquake["geometry"]["coordinates"][1]
            depth1 = earthquake["geometry"]["coordinates"][2]
            mag = earthquake["properties"]["mag"]
            datetime1 = earthquake["properties"]["time"]
            ctry_code = rg.search((lat,lon))[0]["cc"]
            new_earthquake = Earthquakes(EarthquakeID=earthquake["id"], Longitude=lon, Latitude=lat, Depth=depth1, DateTime=datetime1, Magnitude=mag, country_id=ctry_code)

            # Add new_earthquake to the current session
            session.add(new_earthquake)

            # Commit objects to the database
            session.commit()
            
        except:
            print(f"{earthquake['id']}, {lon}, {lat}, {depth1}, {earthquake['properties']['time']}, {ctry_code}")
        
        cont += 1
    else:
        break


conn.close()
engine.dispose()

  0%|          | 0/9157 [00:00<?, ?it/s]

Loading formatted geocoded file...


  0%|          | 10/9157 [00:19<4:55:05,  1.94s/it]


In [19]:
print("DATABASE COMPLETED")

DATABASE COMPLETED
